In [2]:
import pandas as pd
import numpy as np
from collections import Counter
import nltk
import pandas as pd
#from emoticons import EmoticonDetector
import re as regex
import numpy as np
#import plotly
#from plotly import graph_objs
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from time import time
import matplotlib.pyplot as plt
%matplotlib inline
#plotly.offline.init_notebook_mode()

import seaborn as sns
#import plotly
#import cufflinks as cf
import re
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Amar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Import the Data

In [3]:
train_data = pd.read_csv('Train.csv')
test_data = pd.read_csv('Test.csv')

In [4]:
train_data.head()

,id,original_text,lang,retweet_count,original_author,sentiment_class
0,1.245025e+18,Happy #MothersDay to all you amazing mothers o...,en,0,BeenXXPired,0
1,1.245759e+18,Happy Mothers Day Mum - I'm sorry I can't be t...,en,1,FestiveFeeling,0
2,1.246087e+18,Happy mothers day To all This doing a mothers ...,en,0,KrisAllenSak,-1
3,1.244803e+18,Happy mothers day to this beautiful woman...ro...,en,0,Queenuchee,0
4,1.244876e+18,Remembering the 3 most amazing ladies who made...,en,0,brittan17446794,-1


In [5]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3235 entries, 0 to 3234
Data columns (total 6 columns):
id                 3235 non-null float64
original_text      3235 non-null object
lang               3231 non-null object
retweet_count      3231 non-null object
original_author    3235 non-null object
sentiment_class    3235 non-null int64
dtypes: float64(1), int64(1), object(4)
memory usage: 151.8+ KB


In [6]:
test_data.head()

,id,original_text,lang,retweet_count,original_author
0,1.246628e+18,"3. Yeah, I once cooked potatoes when I was 3 y...",en,0,LToddWood
1,1.245898e+18,"Happy Mother's Day to all the mums, step-mums,...",en,0,iiarushii
2,1.244717e+18,"I love the people from the UK, however, when I...",en,0,andreaanderegg
3,1.245730e+18,Happy 81st Birthday Happy Mother’s Day to my m...,en,1,TheBookTweeters
4,1.244636e+18,Happy Mothers day to all those wonderful mothe...,en,0,andreaanderegg


In [7]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1387 entries, 0 to 1386
Data columns (total 5 columns):
id                 1387 non-null float64
original_text      1387 non-null object
lang               1387 non-null object
retweet_count      1386 non-null object
original_author    1387 non-null object
dtypes: float64(1), object(4)
memory usage: 54.3+ KB


# Data Cleaning

In [8]:
def clean_tweets(tweet):
    
    # remove URL
    tweet = re.sub(r"http\S+", "", tweet)
    
    # Remove usernames
    tweet = re.sub(r"@[^\s]+[\s]?",'',tweet)
    
    # remove special characters 
    tweet = re.sub('[^ a-zA-Z0-9]', '', tweet)
    
    # remove Numbers
    tweet = re.sub('[0-9]', '', tweet)
    
    return tweet

In [9]:
train_data['original_text'] = train_data['original_text'].apply(clean_tweets)

In [10]:
train_data['original_text'].head()

0    Happy MothersDay to all you amazing mothers ou...
1    Happy Mothers Day Mum  Im sorry I cant be ther...
2    Happy mothers day To all This doing a mothers ...
3    Happy mothers day to this beautiful womanroyal...
4    Remembering the  most amazing ladies who made ...
Name: original_text, dtype: object

# Tokenization & stemming

In [11]:
# Function which directly tokenize the tweet data
from nltk.tokenize import TweetTokenizer

tt = TweetTokenizer()
train_data['original_text'].apply(tt.tokenize)

0       [Happy, MothersDay, to, all, you, amazing, mot...
1       [Happy, Mothers, Day, Mum, Im, sorry, I, cant,...
2       [Happy, mothers, day, To, all, This, doing, a,...
3       [Happy, mothers, day, to, this, beautiful, wom...
4       [Remembering, the, most, amazing, ladies, who,...
                              ...                        
3230    [To, all, my, sisters, my, sisters, in, law, a...
3231    [Happy, Mothers, Day, to, all, the, Mums, Step...
3232    [Happy, Mothers, Day, to, the, craziest, woman...
3233    [Happy, Mothers, Day, to, my, amazing, wife, W...
3234    [Wishing, you, all, a, safe, happy, Mothers, D...
Name: original_text, Length: 3235, dtype: object

In [12]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

ps = PorterStemmer()

In [13]:
def tokenize(text):
    return word_tokenize(text)

def stemming(words):
    stem_words = []
    for w in words:
        w = ps.stem(w)
        stem_words.append(w)
    
    return stem_words

In [14]:
# apply tokenize function
train_data['text'] = train_data['original_text'].apply(tokenize)

In [15]:
# apply steming function
train_data['tokenized'] = train_data['text'].apply(stemming)

In [16]:
train_data.head()

,id,original_text,lang,retweet_count,original_author,sentiment_class,text,tokenized
0,1.245025e+18,Happy MothersDay to all you amazing mothers ou...,en,0,BeenXXPired,0,"[Happy, MothersDay, to, all, you, amazing, mot...","[happi, mothersday, to, all, you, amaz, mother..."
1,1.245759e+18,Happy Mothers Day Mum Im sorry I cant be ther...,en,1,FestiveFeeling,0,"[Happy, Mothers, Day, Mum, Im, sorry, I, cant,...","[happi, mother, day, mum, Im, sorri, I, cant, ..."
2,1.246087e+18,Happy mothers day To all This doing a mothers ...,en,0,KrisAllenSak,-1,"[Happy, mothers, day, To, all, This, doing, a,...","[happi, mother, day, To, all, thi, do, a, moth..."
3,1.244803e+18,Happy mothers day to this beautiful womanroyal...,en,0,Queenuchee,0,"[Happy, mothers, day, to, this, beautiful, wom...","[happi, mother, day, to, thi, beauti, womanroy..."
4,1.244876e+18,Remembering the most amazing ladies who made ...,en,0,brittan17446794,-1,"[Remembering, the, most, amazing, ladies, who,...","[rememb, the, most, amaz, ladi, who, made, me,..."


# Wordlist

In [17]:
words = Counter()
for idx in train_data.index:
    words.update(train_data.loc[idx, "text"])

words.most_common(5)

[('to', 3673),
 ('the', 2850),
 ('Happy', 2738),
 ('and', 2445),
 ('Mothers', 2379)]

In [18]:
nltk.download('stopwords')
stopwords=nltk.corpus.stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Amar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
whitelist = ["n't", "not"]
for idx, stop_word in enumerate(stopwords):
    if stop_word not in whitelist:
        del words[stop_word]
words.most_common(5)

[('Happy', 2738),
 ('Mothers', 2379),
 ('Day', 1964),
 ('day', 1606),
 ('mothers', 1431)]

In [23]:
def word_list(processed_data):
    #print(processed_data)
    min_occurrences=3 
    max_occurences=500 
    stopwords=nltk.corpus.stopwords.words("english")
    whitelist = ["n't","not"]
    wordlist = []
    
    whitelist = whitelist if whitelist is None else whitelist
    #print(whitelist)
    '''
    import os
    if os.path.isfile("wordlist.csv"):
        word_df = pd.read_csv("wordlist.csv")
        word_df = word_df[word_df["occurrences"] > min_occurrences]
        wordlist = list(word_df.loc[:, "word"])
        #return
    '''
    words = Counter()
    for idx in processed_data.index:
        words.update(processed_data.loc[idx, "text"])

    for idx, stop_word in enumerate(stopwords):
        if stop_word not in whitelist:
            del words[stop_word]
    #print(words)

    word_df = pd.DataFrame(data={"word": [k for k, v in words.most_common() if min_occurrences < v < max_occurences],
                                 "occurrences": [v for k, v in words.most_common() if min_occurrences < v < max_occurences]},
                           columns=["word", "occurrences"])
    #print(word_df)
    word_df.to_csv("wordlist.csv", index_label="idx")
    wordlist = [k for k, v in words.most_common() if min_occurrences < v < max_occurences]
    #print(wordlist)

In [24]:
word_list(train_data)

In [26]:
words = pd.read_csv("wordlist.csv")

# Bag of Words

In [27]:
import os

In [28]:
wordlist= []
if os.path.isfile("wordlist.csv"):
    word_df = pd.read_csv("wordlist.csv")
    word_df = word_df[word_df["occurrences"] > 3]
    wordlist = list(word_df.loc[:, "word"])

label_column = ["label"]
columns = label_column + list(map(lambda word: str(word) + '_bow',wordlist))
labels = []
rows = []
for idx in train_data.index:
    current_row = []
    
    # add label
    current_label = train_data.loc[idx, "sentiment_class"]
    labels.append(current_label)
    current_row.append(current_label)

    # add bag-of-words
    tokens = set(train_data.loc[idx, "text"])
    for _, word in enumerate(wordlist):
        current_row.append(1 if word in tokens else 0)

    rows.append(current_row)

data_model = pd.DataFrame(rows, columns=columns)
data_labels = pd.Series(labels)


bow = data_model

In [29]:
bow.head()

,label,today_bow,us_bow,We_bow,mother_bow,amazing_bow,not_bow,wonderful_bow,Mum_bow,To_bow,...,black_bow,Maybe_bow,gem_bow,werent_bow,Markel_bow,countries_bow,calling_bow,Prince_bow,practicing_bow,calls_bow
0,0,1,1,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,-1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,-1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Classification

In [30]:
import random
seed = 777
random.seed(seed)

In [33]:
def test_classifier(X_train, y_train, X_test, y_test, classifier):
    log("")
    log("---------------------------------------------------------")
    log("Testing " + str(type(classifier).__name__))
    now = time()
    list_of_labels = sorted(list(set(y_train)))
    model = classifier.fit(X_train, y_train)
    log("Learing time {0}s".format(time() - now))
    now = time()
    predictions = model.predict(X_test)
    log("Predicting time {0}s".format(time() - now))

    # Calculate Accuracy, Precision, recall
    
    precision = precision_score(y_test, predictions, average=None, pos_label=None, labels=list_of_labels)
    recall = recall_score(y_test, predictions, average=None, pos_label=None, labels=list_of_labels)
    accuracy = accuracy_score(y_test, predictions)
    f1 = f1_score(y_test, predictions, average=None, pos_label=None, labels=list_of_labels)
    
    log("=================== Results ===================")
    log("            Negative     Neutral     Positive")
    log("F1       " + str(f1))
    log("Precision" + str(precision))
    log("Recall   " + str(recall))
    log("Accuracy " + str(accuracy))
    log("===============================================")

    return precision, recall, accuracy, f1

def log(x):
    #can be used to write to log file
    print(x)

# Experiment 1: BOW + Naive Bayes

In [34]:
from sklearn.naive_bayes import BernoulliNB
X_train, X_test, y_train, y_test = train_test_split(bow.iloc[:, 1:], bow['label'], test_size=0.3, random_state=seed)
precision, recall, accuracy, f1 = test_classifier(X_train, y_train, X_test, y_test, BernoulliNB())


---------------------------------------------------------
Testing BernoulliNB
Learing time 0.48799610137939453s
Predicting time 0.14699816703796387s
=================== Results ===================
            Negative     Neutral     Positive
F1       [0.1627907  0.6193969  0.20485175]
Precision[0.24778761 0.52054795 0.296875  ]
Recall   [0.12121212 0.76458753 0.1563786 ]
Accuracy 0.4593202883625129


In [35]:
def cv(classifier, X_train, y_train):
    log("===============================================")
    classifier_name = str(type(classifier).__name__)
    now = time()
    log("Crossvalidating " + classifier_name + "...")
    accuracy = [cross_val_score(classifier, X_train, y_train, cv=8, n_jobs=-1)]
    log("Crosvalidation completed in {0}s".format(time() - now))
    log("Accuracy: " + str(accuracy[0]))
    log("Average accuracy: " + str(np.array(accuracy[0]).mean()))
    log("===============================================")
    return accuracy

# Add Extra Features

In [41]:
train_data = pd.read_csv('Train.csv')
test_data = pd.read_csv('Test.csv')

In [42]:
def add_extra_feature(df, tweet_column):
    
    # Print Number of Exclamation
    #length_of_excl = (len(re.findall(r'!', string)))
    df['number_of_exclamation'] = tweet_column.apply(lambda x: (len(re.findall(r'!', x))))
    
    # Number of ?
    #length_of_questionmark = (len(re.findall(r'?', string)))
    df['number_of_questionmark'] = tweet_column.apply(lambda x: (len(re.findall(r'[?]', x))))
    
    # Number of #
    df['number_of_hashtag'] = tweet_column.apply(lambda x: (len(re.findall(r'#', x))))
    
    # Number of @
    df['number_of_mention'] = tweet_column.apply(lambda x: (len(re.findall(r'@', x))))
    
    # Number of Quotes
    df['number_of_quotes'] = tweet_column.apply(lambda x: (len(re.findall(r"'", x))))

    # Number if underscore
    df['number_of_underscore'] = tweet_column.apply(lambda x: (len(re.findall(r'_', x))))
    
    
    #print((txt.split(" "), row))
    #print(row.split())

In [43]:
# pass the train_data into add_extra_feature function
add_extra_feature(train_data, train_data["original_text"])

# ADD EMOTICONS

In [45]:
## Emoticon Detector

class EmoticonDetector:
    emoticons = {}

    def __init__(self, emoticon_file="emoticons.txt"):
        from pathlib import Path
        content = Path(emoticon_file).read_text()
        positive = True
        for line in content.split("\n"):
            if "positive" in line.lower():
                positive = True
                continue
            elif "negative" in line.lower():
                positive = False
                continue

            self.emoticons[line] = positive

    def is_positive(self, emoticon):
        if emoticon in self.emoticons:
            return self.emoticons[emoticon]
        return False

    def is_emoticon(self, to_check):
        return to_check in self.emoticons

In [47]:
ed = EmoticonDetector()

In [48]:
processed_data = train_data.copy()

def add_column(column_name, column_content):
    processed_data.loc[:, column_name] = pd.Series(column_content, index=processed_data.index)

def count_by_lambda(expression, word_array):
    return len(list(filter(expression, word_array)))

add_column("splitted_text", map(lambda txt: txt.split(" "), processed_data["original_text"]))

positive_emo = list(
    map(lambda txt: count_by_lambda(lambda word: ed.is_emoticon(word) and ed.is_positive(word), txt),
        processed_data["splitted_text"]))
add_column("number_of_positive_emo", positive_emo)

negative_emo = list(map(
    lambda txt: count_by_lambda(lambda word: ed.is_emoticon(word) and not ed.is_positive(word), txt),
    processed_data["splitted_text"]))

add_column("number_of_negative_emo", negative_emo)

In [49]:
train_data['number_of_positive_emo'] = positive_emo
train_data['number_of_negative_emo'] = negative_emo

# Prepare training data for model

In [50]:
train_data.head()

,id,original_text,lang,retweet_count,original_author,sentiment_class,number_of_exclamation,number_of_questionmark,number_of_hashtag,number_of_mention,number_of_quotes,number_of_underscore,number_of_positive_emo,number_of_negative_emo
0,1.245025e+18,Happy #MothersDay to all you amazing mothers o...,en,0,BeenXXPired,0,1,0,2,0,2,0,0,0
1,1.245759e+18,Happy Mothers Day Mum - I'm sorry I can't be t...,en,1,FestiveFeeling,0,1,0,0,0,4,0,0,0
2,1.246087e+18,Happy mothers day To all This doing a mothers ...,en,0,KrisAllenSak,-1,0,0,1,0,0,0,0,0
3,1.244803e+18,Happy mothers day to this beautiful woman...ro...,en,0,Queenuchee,0,0,0,2,0,0,0,0,0
4,1.244876e+18,Remembering the 3 most amazing ladies who made...,en,0,brittan17446794,-1,3,0,0,0,0,0,0,0


In [51]:
# apply the clean tweet function
train_data['original_text'] = train_data['original_text'].apply(clean_tweets)

In [52]:
## Tokenize & stemming data
train_data['text'] = train_data['original_text'].apply(tokenize)
train_data['tokenized'] = train_data['text'].apply(stemming)

In [59]:
wordlist= []
if os.path.isfile("wordlist.csv"):
    word_df = pd.read_csv("wordlist.csv")
    word_df = word_df[word_df["occurrences"] > 3]
    wordlist = list(word_df.loc[:, "word"])

label_column = ["label"]
columns = label_column + list(map(lambda w: str(w) + "_bow",wordlist))
labels = []
rows = []
for idx in train_data.index:
    current_row = []
        # add label
    current_label = train_data.loc[idx, "sentiment_class"]
    labels.append(current_label)
    current_row.append(current_label)

    # add bag-of-words
    tokens = set(train_data.loc[idx, "text"])
    for _, word in enumerate(wordlist):
        current_row.append(1 if word in tokens else 0)

    rows.append(current_row)

data_model = pd.DataFrame(rows, columns=columns)
data_labels = pd.Series(labels)

In [60]:
dat1 = train_data
dat2 = data_model

dat1 = dat1.reset_index(drop=True)
dat2 = dat2.reset_index(drop=True)

data_model = dat1.join(dat2)

In [61]:
train_data.columns

Index(['id', 'original_text', 'lang', 'retweet_count', 'original_author',
       'sentiment_class', 'number_of_exclamation', 'number_of_questionmark',
       'number_of_hashtag', 'number_of_mention', 'number_of_quotes',
       'number_of_underscore', 'number_of_positive_emo',
       'number_of_negative_emo', 'text', 'tokenized'],
      dtype='object')

In [62]:
## Drop the columns in data_model
data_model = data_model.drop(columns=['id', 'original_text', 'lang', 'retweet_count', 'original_author',
       'sentiment_class', 'text', 'tokenized'], axis=1)

In [63]:
data_model.columns

Index(['number_of_exclamation', 'number_of_questionmark', 'number_of_hashtag',
       'number_of_mention', 'number_of_quotes', 'number_of_underscore',
       'number_of_positive_emo', 'number_of_negative_emo', 'label',
       'today_bow',
       ...
       'black_bow', 'Maybe_bow', 'gem_bow', 'werent_bow', 'Markel_bow',
       'countries_bow', 'calling_bow', 'Prince_bow', 'practicing_bow',
       'calls_bow'],
      dtype='object', length=1827)

# Experiment 2: Added feature + Random Forest

In [64]:
from sklearn.ensemble import RandomForestClassifier
X_train, X_test, y_train, y_test = train_test_split(data_model.drop(columns='label',axis=1),data_model['label'] , test_size=0.3)
precision, recall, accuracy, f1 = test_classifier(X_train, y_train, X_test, y_test, RandomForestClassifier(random_state=seed,n_estimators=403,n_jobs=-1))


---------------------------------------------------------
Testing RandomForestClassifier
Learing time 17.766369581222534s
Predicting time 0.534226655960083s
=================== Results ===================
            Negative     Neutral     Positive
F1       [0.14634146 0.68181818 0.11683849]
Precision[0.328125   0.55357143 0.25373134]
Recall   [0.0941704  0.88740458 0.07589286]
Accuracy 0.518022657054583


In [65]:
rf_acc = cv(RandomForestClassifier(n_estimators=403,n_jobs=-1, random_state=seed),data_model.drop(columns='label',axis=1), data_model['label'])

Crossvalidating RandomForestClassifier...
Crosvalidation completed in 199.65104413032532s
Accuracy: [0.48275862 0.52592593 0.47654321 0.49382716 0.48641975 0.51861042
 0.51116625 0.49875931]
Average accuracy: 0.49925133127765686


# Experiment 3: Added Feature + XGBoost

In [67]:
!pip install xgboost
from xgboost import XGBClassifier as XGBoostClassifier

In [69]:
X_train, X_test, y_train, y_test = train_test_split(data_model.drop(columns='label',axis=1),data_model['label'] , test_size=0.3)
precision, recall, accuracy, f1 = test_classifier(X_train, y_train, X_test, y_test, XGBoostClassifier(seed=seed))


---------------------------------------------------------
Testing XGBClassifier
Learing time 108.92083263397217s
Predicting time 0.4709954261779785s
=================== Results ===================
            Negative     Neutral     Positive
F1       [0.15238095 0.63741339 0.12804878]
Precision[0.26373626 0.53626943 0.19444444]
Recall   [0.10714286 0.78557875 0.09545455]
Accuracy 0.4727085478887745


In [71]:
xgb_acc = cv(XGBoostClassifier(seed=seed),data_model.drop(columns='label',axis=1), data_model['label'])

Crossvalidating XGBClassifier...
Crosvalidation completed in 954.9081835746765s
Accuracy: [0.48029557 0.50617284 0.47407407 0.4962963  0.49135802 0.46898263
 0.49131514 0.46650124]
Average accuracy: 0.4843744760643166


# Experiment 4: Added Feature + Naive Bayes

In [70]:
X_train, X_test, y_train, y_test = train_test_split(data_model.drop(columns='label',axis=1),data_model['label'] , test_size=0.3)
precision, recall, accuracy, f1 = test_classifier(X_train, y_train, X_test, y_test, BernoulliNB())


---------------------------------------------------------
Testing BernoulliNB
Learing time 0.5520036220550537s
Predicting time 0.15999817848205566s
=================== Results ===================
            Negative     Neutral     Positive
F1       [0.16976127 0.58959538 0.17514124]
Precision[0.22535211 0.5        0.26956522]
Recall   [0.13617021 0.71830986 0.12970711]
Accuracy 0.4325437693099897


In [72]:
nb_acc = cv(BernoulliNB(), data_model.drop(columns='label',axis=1), data_model['label'])

Crossvalidating BernoulliNB...
Crosvalidation completed in 13.766727209091187s
Accuracy: [0.45320197 0.44691358 0.43209877 0.47407407 0.41728395 0.44416873
 0.42183623 0.43424318]
Average accuracy: 0.44047755997144206


In [73]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            log("Model with rank: {0}".format(i))
            log("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                results['mean_test_score'][candidate],
                results['std_test_score'][candidate]))
            log("Parameters: {0}".format(results['params'][candidate]))
            log("")

def best_fit(X_train, y_train, n_iter=5):
    
    parameters = {
        "n_estimators":[103,201, 403],
        "max_depth":[3,10,15, 30],
        "objective":["multi:softmax","binary:logistic"],
        "learning_rate":[0.05, 0.1, 0.15, 0.3]
    }

    rand_search = RandomizedSearchCV(XGBoostClassifier(seed=seed),param_distributions=parameters,
                                     n_iter=n_iter,scoring="accuracy",
                                     n_jobs=-1,cv=8)

    import time as ttt
    now = time()
    log(ttt.ctime())
    rand_search.fit(X_train, y_train)
    report(rand_search.cv_results_, 10)
    log(ttt.ctime())
    log("Search took: " + str(time() - now))

In [ ]:
best_fit(data_model.drop(columns='label',axis=1), data_model['label'], n_iter=10)

Tue Jun  9 19:21:50 2020
